In [4]:
from pyvi import ViTokenizer, ViPosTagger
import numpy as np
import pandas as pd
# import gensim
import sklearn
import tensorflow as tf
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.layers import *
from keras.layers import *
from keras.preprocessing.text import Tokenizer
from keras.models import Model
from keras.metrics import sparse_categorical_accuracy
from tensorflow.keras.optimizers import Adam

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
!pip install pyvi
!pip install py_vncorenlp

In [ ]:
datanewscontent = pd.read_csv("data.csv",on_bad_lines='skip')
datanewscontent.head()

,data,label
0,JetNation FanDuel League; Tuần 4,1
1,Tổng thống Hoa Kỳ Donald Trump lặng lẽ ký luật...,1
2,Bảng xếp hạng Đội phòng thủ trong Bóng đá Ảo 2...,1
3,Chỉ cần im lặng và chơi một số bóng chày chết ...,1
4,"10/7: Chuck Axed; HBD Brickyard, Adam, Moonlig...",1


In [ ]:
datanew=[]
for d in datanewscontent['data']:
    e=ViTokenizer.tokenize(str(d))
    datanew.append(e)


In [ ]:
def truncatedvectors(data,n_components=300):
  svd_ngram = TruncatedSVD(n_components=n_components, random_state=42)
  svd_ngram.fit(data)
  return svd_ngram.transform(data)


In [ ]:
def tfidf(data):
  tfidf_vect_ngram = TfidfVectorizer(analyzer='word', max_features=30000, ngram_range=(1, 2))
  tfidf_vect_ngram.fit(data)
  X_data_tfidf_ngram =  tfidf_vect_ngram.transform(data)
  return truncatedvectors(X_data_tfidf_ngram)

In [ ]:
import string
from py_vncorenlp import VnCoreNLP
import re
def clean_text(text):
    text = re.sub('<.*?>', '', text).strip()
    text = re.sub('(\s)+', r'\1', text)
    for ele in text.split():
      if ele.isdigit():
          text = text.replace(ele, "@")
    for character in text:
      if character.isdigit():
          text = text.replace(character, "@")
    chars = re.escape(string.punctuation)
    text = re.sub(r'['+chars+']', '', text)
    text = text.lower()
    return text


In [ ]:
datanewscont = datanewscontent["data"] = datanewscontent["data"].apply(clean_text)
X=tfidf(datanewscont)
Y = datanewscontent["label"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.linear_model import PassiveAggressiveClassifier
PAC = PassiveAggressiveClassifier(random_state=0)
PAC.fit(x_train,y_train)
pred_PAC=PAC.predict(x_test)
PAC.score(x_test, y_test)
score = accuracy_score(y_test, pred_PAC)
print(classification_report(y_test, pred_PAC))
print(f'Accuracy: {round(score*100, 2)}%')


              precision    recall  f1-score   support

           0       0.97      0.88      0.93       419
           1       0.90      0.98      0.94       461

    accuracy                           0.93       880
   macro avg       0.94      0.93      0.93       880
weighted avg       0.94      0.93      0.93       880

Accuracy: 93.3%


In [ ]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(random_state=0)
RFC.fit(x_train, y_train)
pred_rfc = RFC.predict(x_test)
RFC.score(x_test, y_test)
score = accuracy_score(y_test, pred_rfc)
print(classification_report(y_test, pred_rfc))
print(f'Accuracy: {round(score*100, 2)}%')

              precision    recall  f1-score   support

           0       0.87      0.97      0.92       419
           1       0.97      0.87      0.92       461

    accuracy                           0.92       880
   macro avg       0.92      0.92      0.92       880
weighted avg       0.92      0.92      0.92       880

Accuracy: 91.82%


In [ ]:
def manual_testing(news):
  testing_news = {"data":[news]}
  new_def_test = pd.DataFrame(testing_news)
  new_def_test["data"] = new_def_test["data"].apply(clean_text) 
  new_x_test = new_def_test["data"]
  new_xv_test = tfidf([new_x_test])
  classification = PAC.predict(new_xv_test)


In [ ]:
import pickle
pickle.dump(PAC,open('model.pkl', 'wb'))

In [ ]:
# load the model from disk
loaded_model = pickle.load(open('model.pkl', 'rb'))

In [ ]:
news = str(input())
manual_testing(news)


OK


AttributeError: ignored

In [2]:
import pandas as pd
df = pd.read_csv('dataset_artical_real.csv',on_bad_lines='skip')
df.head(200)

,Title,Content,Link,Date,Label
0,Một phụ nữ bịt mặt bắt cóc cháu bé 5 tuổi bị n...,"Ngày 6.5, Công an thị xã Phước Long, tỉnh Bình...",https://laodong.vn/phap-luat/mot-phu-nu-bit-ma...,2018-05-06 18:27:52,1.0
1,Tung tin Mr Bean qua đời để phát tán virus độc...,"Hôm 19/7, nhiều độc giả cảnh báo nhau trên mạn...",https://news.zing.vn/tung-tin-mr-bean-qua-doi-...,2018-07-19 15:18,1.0
2,Mưa lớn do bão Usagi gây ngập úng 70 ha rau ở ...,"Ngày 27/11, Ban Chỉ huy Phòng chống thiên tai ...",https://vnexpress.net/mua-lon-do-bao-usagi-gay...,2018-11-26 00:00:00,1.0
3,"Xả súng tại nhà thờ Mỹ, ít nhất 26 người thiệt...",Cảnh sát phong tỏa bên ngoài nhà thờ nơi xảy r...,https://vnexpress.net/xa-sung-tai-nha-tho-my-i...,2017-11-06 00:00:00,1.0
4,HLV Eriksson dự đoán tuyển VN sẽ đá bại Malaysia,Ông Sven Goran Eriksson - Theo trang web chí...,https://tuoitre.vn/hlv-eriksson-du-doan-tuyen-...,2018-12-09 00:00:00,1.0
...,...,...,...,...,...
95,Không có chuyện thí điểm chữ viết 'Tiếq Việt' ...,"Ông Phạm Ngọc Thanh, Phó giám đốc Sở GD&ĐT TP....",https://news.zing.vn/khong-co-chuyen-thi-diem-...,2017-12-01 14:44:00,1.0
96,Bộ Công Thương kết luận Con Cưng có sai sót nh...,"Kết luận về vụ Con Cưng, Bộ Công Thương cho bi...",https://news.zing.vn/bo-cong-thuong-ket-luan-c...,2018-08-17 12:31:00,1.0
97,Phát hiện văn phòng công chứng giả tại TP.HCM,"600 vụ việc được công chứng, chứng thực từ văn...",https://thanhnien.vn/thoi-su/phat-hien-van-pho...,2018-09-29 10:05:12,1.0
98,"Thông ngã trên đèo Prenn, giao thông tắc nghẽn...",Cây thông chắn ngang đường. Ảnh: Duy Khôi. 21...,https://vnexpress.net/tin-tuc/thoi-su/thong-ng...,2018-09-29 00:00:00,1.0
